In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno

In [3]:
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type        to reduce memory usage.            """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                #if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                #    df[col] = df[col].astype(np.float16)
                #elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                #    df[col] = df[col].astype(np.float32)
                #else:
                df[col] = df[col].astype(np.float16)
        else:
            df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    
    return df


In [4]:
df=pd.read_csv("train.csv")
target=df['target']
df1=df.drop('target',axis='columns')


In [5]:
df_test=pd.read_csv("test.csv")
print(df1.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70000 entries, 0 to 69999
Columns: 508 entries, id to feature_506
dtypes: float64(143), int64(273), object(92)
memory usage: 271.3+ MB
None


In [6]:
print('train_shape',df1.shape)
print('test_shape',df_test.shape)

train_shape (70000, 508)
test_shape (30000, 508)


In [7]:
# SPLITTING THE OBJECT AND INT COLUMNS
object_columns_train=df1.select_dtypes(include='object')
object_columns_test=df_test.select_dtypes(include='object')


In [8]:
df2_train=df1.drop(object_columns_train,axis='columns')
df2_test=df_test.drop(object_columns_test,axis='columns')


In [9]:
# REMOVING COLUMNS WHICH HAS MORE THAN 50% NAN VALUES
def removemorethan50pernavalues(dataframe):
    for col in dataframe.columns:
        if (dataframe[col].isna().sum()/dataframe.shape[0]>0.50):
            dataframe.drop(col,axis='columns',inplace=True)
df3_train=df2_train.copy()
df3_test=df2_test.copy()


In [10]:
removemorethan50pernavalues(df3_train)
removemorethan50pernavalues(df3_test)
df4_train=df3_train.copy()
df4_test=df3_test.copy()
print(df4_train.shape)
print(df4_test.shape)
## REMOVING ZERO VARIANCE COLUMNS
zero_var=[]
for col in df4_train.columns:
        if (len(df4_train[col].unique()))==1:
            zero_var.append(col)


(70000, 396)
(30000, 396)


In [11]:
df44_train=df4_train.drop(zero_var,axis='columns')
df44_test=df4_test.drop(zero_var,axis='columns')


In [11]:
#def removezerovariancecolumns(dataframe):
    #for col in dataframe.columns:
        #if (len(dataframe[col].unique()))==1:
            #dataframe.drop(col,axis='columns',inplace=True)
            

In [12]:
#removezerovariancecolumns(df4_train)
#removezerovariancecolumns(df4_test)
df5_train=df44_train.copy()
df5_test=df44_test.copy()


In [13]:
#replacing nan values with mode for col with unique values less than 3
for col in df5_train.columns:
    if (len(df5_train[col].unique()))<3:
        df5_train[col].fillna(df5_train[col].mode(),inplace=True)
for col in df5_test.columns:
    if (len(df5_test[col].unique()))<3:
        df5_test[col].fillna(df5_test[col].mode(),inplace=True)
df6_train=df5_train.copy()
df6_test=df5_test.copy()

In [14]:
#replacing nan values with median for col with unique values more than 4
for col in df6_train.columns:
    if(len(df6_train[col].unique()))>3:
        df6_train[col].fillna(df6_train[col].median(),inplace=True)
for col in df6_test.columns:
    if(len(df6_test[col].unique()))>3:
        df6_test[col].fillna(df6_test[col].median(),inplace=True)
df7_train=df6_train.copy()
df7_test=df6_test.copy()
ID_train=df7_train['id']
ID_test=df7_test['id']
df77_train=df7_train.drop('id',axis='columns')
df77_test=df7_test.drop('id',axis='columns')

df7_test

,id,feature_14,feature_15,feature_25,feature_26,feature_35,feature_36,feature_37,feature_38,feature_39,...,feature_490,feature_492,feature_494,feature_496,feature_497,feature_499,feature_501,feature_503,feature_504,feature_506
0,5,1.758,15.018333,3549,22515,-100.000000,100.000000,-22.837156,-13.821311,-100.000000,...,0,0,0,0,0,0,0,0,0,0
1,7,0.000,0.000000,0,0,-100.000000,147.361188,47.191772,10.499621,-100.000000,...,0,0,0,0,0,0,0,0,0,0
2,9,0.000,0.000000,0,0,-21.087787,140.228572,1.122803,295.047825,-20.201758,...,0,0,0,0,0,0,0,0,0,0
3,11,0.000,0.000000,8841,28896,0.000000,0.000000,18.582679,150.159626,0.000000,...,0,0,0,0,0,0,0,0,0,0
4,14,0.000,0.000000,0,0,0.000000,-100.000000,-34.420654,242.227486,0.000000,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29995,99981,0.000,0.000000,8351,91413,0.000000,0.000000,-2.998692,37.908906,0.000000,...,0,0,0,0,0,0,0,0,0,0
29996,99982,0.000,0.000000,271,390,100.000000,100.000000,-30.700138,-20.344487,100.000000,...,0,0,0,0,0,0,0,0,0,0
29997,99992,0.000,0.249000,5655,51403,-96.920542,138.281865,793.082314,267.236353,-72.497909,...,0,0,0,0,0,0,0,0,0,0
29998,99993,0.000,0.000000,0,0,-21.716540,-28.212716,0.217039,-0.039371,-21.546634,...,0,0,0,0,0,0,0,0,0,0


In [15]:
#df10_train=pd.DataFrame(df9_train)
#df10_test=pd.DataFrame(df9_test)
df10_train=df77_train.copy()
df10_test=df77_test.copy()


In [16]:
# REMOVING OBJECT COLUMNS WITH MORE THAN 50% NAN VALUES
removemorethan50pernavalues(object_columns_train)
removemorethan50pernavalues(object_columns_test)
ob_train=object_columns_train.copy()
ob_test=object_columns_test.copy()

In [17]:
# CONVERTING DATETIME FEATURES
datetime_features = ["feature_191", "feature_192","feature_199", "feature_201"]
def convertingdatetime(dataframe):
    for i in datetime_features:
        dataframe[i]=pd.to_datetime(dataframe[i])
convertingdatetime(ob_train)
convertingdatetime(ob_test)
ob1_train=ob_train.copy()
ob1_test=ob_test.copy()

In [18]:
def convertingtoymd(dataframe):
    for col in dataframe:
        if dataframe[col].dtype=='<M8[ns]':
            dataframe[f"{col}year"] = pd.to_datetime(dataframe[col]).dt.year
            dataframe[f"{col}month"] = pd.to_datetime(dataframe[col]).dt.month
            dataframe[f"{col}day"] = pd.to_datetime(dataframe[col]).dt.day
convertingtoymd(ob1_train)
convertingtoymd(ob1_test)

ob2_train=ob1_train.copy()
ob2_test=ob1_test.copy()

In [19]:
def dropdatetime(dataframe):
    for i in dataframe:
        if dataframe[i].dtype=='<M8[ns]':
            dataframe.drop(i,axis='columns',inplace=True)
dropdatetime(ob2_train)
dropdatetime(ob2_test)
ob3_train=ob2_train.copy()
ob3_test=ob2_test.copy()


In [20]:
#removng cols with more classes
def removecolwithmoreclasses(dataframe):
    for col in dataframe:
        if dataframe[col].dtype=='object':
            if (len(dataframe[col].unique()))>100:
                dataframe.drop(col,axis='columns',inplace=True)
removecolwithmoreclasses(ob3_train)
removecolwithmoreclasses(ob3_test)
ob4_train=ob3_train.copy()
ob4_test=ob3_test.copy()
       

In [21]:
zero_var_obj=[]
for col in ob4_train.columns:
        if (len(ob4_train[col].unique()))==1:
            zero_var_obj.append(col)
zero_var_obj
ob44_train=ob4_train.drop(zero_var_obj,axis='columns')
ob44_test=ob4_test.drop(zero_var_obj,axis='columns')

In [22]:
ob5_train=ob44_train.copy()
ob5_test=ob44_test.copy()
ob5_train


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_16,feature_23,feature_24,feature_27,feature_28,...,feature_191day,feature_192year,feature_192month,feature_192day,feature_199year,feature_199month,feature_199day,feature_201year,feature_201month,feature_201day
0,C0,C0,C1,C5,C11,C0,C0,C19,C0,C0,...,10.0,2017.0,11.0,10.0,2017,11,10,2017.0,11.0,10.0
1,C0,C0,C3,C5,C1,C2,C0,C23,C0,C0,...,25.0,2015.0,3.0,25.0,2017,2,4,2015.0,3.0,25.0
2,C0,C0,C3,C5,C2,C1,C0,C22,C0,C0,...,27.0,2015.0,1.0,27.0,2016,3,24,2015.0,1.0,27.0
3,C0,C0,C1,C5,C1,C0,C0,C2,C0,C0,...,2.0,2014.0,1.0,4.0,2014,1,4,2014.0,1.0,4.0
4,C0,C0,C3,C3,C11,C2,C0,C3,C1,C1,...,20.0,2013.0,10.0,9.0,2013,10,9,2013.0,10.0,9.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69995,C0,C0,C3,C1,C11,C0,C0,C14,C1,C0,...,27.0,2014.0,1.0,2.0,2014,1,2,2014.0,1.0,2.0
69996,C0,C0,C5,C5,C2,C2,C0,C17,C0,C0,...,19.0,2016.0,9.0,19.0,2016,9,19,2016.0,9.0,19.0
69997,C0,C0,C3,C2,C11,C0,C0,C5,C1,C1,...,14.0,2015.0,7.0,25.0,2015,7,25,2015.0,7.0,25.0
69998,C0,C0,C1,C5,C1,C2,C1,C8,C0,C0,...,29.0,2017.0,11.0,29.0,2017,11,29,2017.0,11.0,29.0


In [24]:
# REPLACING THE NAN VALUES FOR OBJECT COLUMNS WITH MODE
for i in ob5_train:
    ob5_train[i].fillna(ob5_train[i].mode()[0],inplace=True)
for i in ob5_test:
    ob5_test[i].fillna(ob5_test[i].mode()[0],inplace=True)

In [25]:
ob6_train=ob5_train.copy()
ob6_test=ob5_test.copy()

In [26]:
print(df10_train.shape)
print(df10_test.shape)
print(ob6_train.shape)
print(ob6_test.shape)

(70000, 342)
(30000, 342)
(70000, 74)
(30000, 74)


In [27]:
# CONCATING THE OBJECT AND INT COLUMNS
frames1=[ob6_train,df10_train]
frames2=[ob6_test,df10_test]

final_train=pd.concat(frames1,axis=1)
final_test=pd.concat(frames2,axis=1)

In [28]:
# ENCODING THE OBJECT COLUMNS
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
def labelencoding(dataframe):
    for i in dataframe:
        if dataframe[i].dtype=='object':
            dataframe[i]=labelencoder.fit_transform(dataframe[i])
labelencoding(final_train)
labelencoding(final_test)

In [29]:
final1_train=final_train.copy()
final1_test=final_test.copy()
X_train=final1_train.copy()
X_test=final1_test.copy()

In [30]:
y_train=target


In [31]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)


(70000, 416)
(30000, 416)
(70000,)


In [32]:
corr=X_train.corrwith(y_train)
corr.sort_values(ascending=False)


feature_307        0.335261
feature_297        0.166057
feature_191year    0.156897
feature_199year    0.154425
feature_192year    0.146387
                     ...   
feature_82        -0.113800
feature_29        -0.114935
feature_110       -0.121336
feature_108       -0.122240
feature_81        -0.157007
Length: 416, dtype: float64

In [33]:
# IMPORTING XGB BOOST
import xgboost as xgb
from xgboost import XGBClassifier


In [34]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
X_train1,X_test1,y_train1,y_test1= train_test_split(X_train,y_train,test_size=0.2)
print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)

(56000, 416)
(14000, 416)
(56000,)
(14000,)


In [35]:
# CONVERTING THE DATE TO XGBOOST ORIGINAL API FOR MODEL TUNING
dtrain = xgb.DMatrix(X_train1, label=y_train1)
dtest = xgb.DMatrix(X_test1, label=y_test1)
drealtrain = xgb.DMatrix(X_train, label=y_train)
drealtest = xgb.DMatrix(X_test)
from sklearn.metrics import accuracy_score


In [37]:
# INTIALIZING THE PARAMETERS
params = {
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1,
    'objective':'binary:logistic',
    'eval_metric':'auc'
}
NUM_BOOST_ROUNDS=999

In [40]:
# FINDING THE BASELINE AUC SCORE
xbg_model1 = xgb.train(
    params,
    dtrain,
    num_boost_round=NUM_BOOST_ROUNDS,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best AUC: {:.2f}, found at round {}".format(
                 xbg_model1.best_score,
                 xbg_model1.best_iteration))

[0]	Test-auc:0.75660
[1]	Test-auc:0.76463
[2]	Test-auc:0.76776
[3]	Test-auc:0.77015
[4]	Test-auc:0.77175
[5]	Test-auc:0.77287
[6]	Test-auc:0.77442
[7]	Test-auc:0.77616
[8]	Test-auc:0.77717
[9]	Test-auc:0.77842
[10]	Test-auc:0.77843
[11]	Test-auc:0.77973
[12]	Test-auc:0.78036
[13]	Test-auc:0.78110
[14]	Test-auc:0.78107
[15]	Test-auc:0.78107
[16]	Test-auc:0.78122
[17]	Test-auc:0.78200
[18]	Test-auc:0.78290
[19]	Test-auc:0.78285
[20]	Test-auc:0.78280
[21]	Test-auc:0.78255
[22]	Test-auc:0.78281
[23]	Test-auc:0.78255
[24]	Test-auc:0.78235
[25]	Test-auc:0.78159
[26]	Test-auc:0.78132
[27]	Test-auc:0.78118
[28]	Test-auc:0.78126
Best AUC: 0.78, found at round 18


In [39]:
# Calculate cross validation
cv_results = xgb.cv(
    params,
    dtrain,
    num_boost_round=NUM_BOOST_ROUNDS,
    seed=42,
    nfold=5,
    metrics={'auc'},
    early_stopping_rounds=10
)
cv_results

,train-auc-mean,train-auc-std,test-auc-mean,test-auc-std
0,0.764386,0.001339,0.751829,0.002812
1,0.773539,0.000469,0.758724,0.003432
2,0.778306,0.001038,0.761771,0.003216
3,0.782969,0.000663,0.763698,0.003182
4,0.786983,0.000523,0.765753,0.003163
5,0.790747,0.000881,0.767345,0.002846
6,0.794328,0.001146,0.769108,0.003151
7,0.798162,0.000776,0.770263,0.003781
8,0.802219,0.000628,0.771488,0.003715
9,0.806012,0.000749,0.772297,0.003747


In [78]:
# FINDING THE BEST VALUES FOR MAX_DEPTH AND MIN_CHILD WEIGHT
gridsearch_params = [
    (max_depth, min_child_weight)
    for max_depth in range(3,8)
    for min_child_weight in range(1,6)
]

# Define initial best params and MAE
max_auc = float("Inf")
best_params = None
for max_depth, min_child_weight in gridsearch_params:
    print("CV with max_depth={}, min_child_weight={}".format(
                             max_depth,
                             min_child_weight))
    # Update our parameters
    params['max_depth'] = max_depth
    params['min_child_weight'] = min_child_weight
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=NUM_BOOST_ROUNDS,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].idxmax()
    print("\tAUC {} for {} rounds\n".format(mean_auc, boost_rounds))
    if mean_auc <max_auc:
        max_auc=mean_auc
        best_params = (max_depth,min_child_weight)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], max_auc))
#print(best_params[0], best_params[1], max_auc)

CV with max_depth=3, min_child_weight=1
	AUC 0.7817244 for 68 rounds

CV with max_depth=3, min_child_weight=2
	AUC 0.78139 for 64 rounds

CV with max_depth=3, min_child_weight=3
	AUC 0.7814186 for 58 rounds

CV with max_depth=3, min_child_weight=4
	AUC 0.7815192000000001 for 52 rounds

CV with max_depth=3, min_child_weight=5
	AUC 0.782334 for 60 rounds

CV with max_depth=4, min_child_weight=1
	AUC 0.7798044 for 26 rounds

CV with max_depth=4, min_child_weight=2
	AUC 0.7802827999999999 for 36 rounds

CV with max_depth=4, min_child_weight=3
	AUC 0.7804448 for 30 rounds

CV with max_depth=4, min_child_weight=4
	AUC 0.7803692 for 37 rounds

CV with max_depth=4, min_child_weight=5
	AUC 0.7813053999999999 for 37 rounds

CV with max_depth=5, min_child_weight=1
	AUC 0.7788413999999999 for 23 rounds

CV with max_depth=5, min_child_weight=2
	AUC 0.7785797999999999 for 27 rounds

CV with max_depth=5, min_child_weight=3
	AUC 0.7785646 for 22 rounds

CV with max_depth=5, min_child_weight=4
	AUC 0.7

In [42]:
params['max_depth'] = 7
params['min_child_weight'] = 3
params

{'max_depth': 7,
 'min_child_weight': 3,
 'eta': 0.3,
 'subsample': 1,
 'colsample_bytree': 1,
 'objective': 'binary:logistic',
 'eval_metric': 'auc'}

In [87]:
# FINDING THE BEST VALUES FOR SUBSAMPLE AND COLSAMPLE
gridsearch_params = [
    (subsample, colsample)
    for subsample in [i/10. for i in range(7,11)]
    for colsample in [i/10. for i in range(7,11)]
]
# Define initial best params and MAE
max_auc = float("Inf")
best_params = None
for subsample, colsample in reversed(gridsearch_params):
    print("CV with subsample={}, colsample={}".format(
                             subsample,
                             colsample))
    # Update our parameters
    params['subsample'] = subsample
    params['colsample_bytree'] = colsample
    # Run CV
    cv_results = xgb.cv(
        params,
        dtrain,
        num_boost_round=NUM_BOOST_ROUNDS,
        seed=42,
        nfold=5,
        metrics={'auc'},
        early_stopping_rounds=10
    )
    # Update best MAE
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].idxmax()
    print("\tAUC {} for {} rounds\n".format(mean_auc, boost_rounds))
    if mean_auc <max_auc:
        max_auc=mean_auc
        best_params = (subsample,colsample)
print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], max_auc))
#print(best_params[0], best_params[1], max_auc)


CV with subsample=1.0, colsample=1.0
[21:59:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:59:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[21:59:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample"

[22:01:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


	AUC 0.7734238 for 16 rounds

CV with subsample=0.9, colsample=1.0
[22:01:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:01:35] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:5

[22:03:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:03:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


	AUC 0.7745962 for 15 rounds

CV with subsample=0.8, colsample=1.0
[22:03:39] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:5

[22:05:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:05:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:05:28] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could b

[22:07:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:07:26] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:07:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could b

In [101]:
# FINDING BEST VALUES FOR LEARNING RATE
min_mae = float("Inf")
best_params = None
for eta in [.3, .2, .1, .05, .01, .005]:
    print("CV with eta={}".format(eta))
    # We update our parameters
    params['eta'] = eta
    # Run and time CV
    cv_results = xgb.cv(
            params,
            dtrain,
            num_boost_round=NUM_BOOST_ROUNDS,
            seed=42,
            nfold=5,
            metrics=['auc'],
            early_stopping_rounds=10
          )
    # Update best score
    mean_auc = cv_results['test-auc-mean'].max()
    boost_rounds = cv_results['test-auc-mean'].idxmax()
    print("\tAUC {} for {} rounds\n".format(mean_auc, boost_rounds))
    if mean_auc <max_auc:
        max_auc=mean_auc
        best_params = eta
print("Best params: {}, AUC: {}".format(best_params, max_auc))

CV with eta=0.3
[22:38:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:38:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:38:09] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

	AUC 0.7828434 for 165 rounds

CV with eta=0.01
[22:45:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:45:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[22:45:54] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { 

KeyboardInterrupt: 

In [43]:
params['subsample'] = 0.7
params['colsample_bytree'] = 0.9
params['eta']=0.05
params

{'max_depth': 7,
 'min_child_weight': 3,
 'eta': 0.05,
 'subsample': 0.7,
 'colsample_bytree': 0.9,
 'objective': 'binary:logistic',
 'eval_metric': 'auc'}

In [116]:
# FINDING OPTIMAL NUMBER OF BOOSTING ROUNDS
xbg_model1 = xgb.train(
    params,
    dtrain,
    num_boost_round=NUM_BOOST_ROUNDS,
    evals=[(dtest, "Test")],
    early_stopping_rounds=10
)
print("Best AUC: {:.2f}, found at round {}".format(
                 xbg_model1.best_score,
                 xbg_model1.best_iteration))

[23:11:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "colsample" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	Test-auc:0.74523
[1]	Test-auc:0.75645
[2]	Test-auc:0.76115
[3]	Test-auc:0.76413
[4]	Test-auc:0.76437
[5]	Test-auc:0.76485
[6]	Test-auc:0.76512
[7]	Test-auc:0.76552
[8]	Test-auc:0.76626
[9]	Test-auc:0.76647
[10]	Test-auc:0.76690
[11]	Test-auc:0.76683
[12]	Test-auc:0.76745
[13]	Test-auc:0.76819
[14]	Test-auc:0.76815
[15]	Test-auc:0.76816
[16]	Test-auc:0.76811
[17]	Test-auc:0.76824
[18]	Test-auc:0.76820
[19]	Test-auc:0.76851
[20]	Test-auc:0.76861
[21]	Test-auc:0.76876
[22]	Test-auc:0.76876
[23]	Test-auc:0.76877
[24]	Test-auc:0.76935
[25]	Test-auc:0.76942
[26]	Test-auc:0.7700

In [44]:
# TRAINING THE XGBOOST MODEL WITH BEST HYPERPARAMETERS
xbg_model3 = xgb.train(
    params,
    drealtrain,
    num_boost_round=128)


In [45]:
# PREDICTING
y_newpred=xbg_model3.predict(drealtest)
p=y_newpred
#p=y_newpred[:,1]
p


array([0.3890527 , 0.27708012, 0.41502   , ..., 0.1605968 , 0.63478816,
       0.10886513], dtype=float32)

In [119]:
# SUBMITTING IT TO KAGGLE
output2 = pd.DataFrame({'id':ID_test,'target':p})
output2.to_csv('netflixxgb5.csv', index=False)
